In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb


In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


def c_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, g_kdl)


def c_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_u2c(q_none, qdot_none)
    
def c_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    c_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot)
    
    
def c_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.NonlinearEffects(rbdlmodel, q_np, qdot_np, c_rbdl)


def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)


def g_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_u2c(q_none)
    
def g_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot, qddot)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, g_rbdl)
    
    
def ID_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    ID_u2c(q_none, qdot_none, qddot_none)
    
    
def ID_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2 
    ID_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot_none)
    
    
def ID_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, ID_rbdl)

    
def M_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToMass(q_kdl, M_kdl)


def M_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    M_u2c(q_none)
    
def M_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    H = pb.calculateMassMatrix(pbmodel, q_none)
    
    
def M_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, M_rbdl)
    
    
def aba_u2c_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    aba_u2c(q_none, qdot_none, tau_none)
    
    
def crba_u2c_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    crba_u2c(q_none, qdot_none, tau_none)
    
    
def fd_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    rbdl.ForwardDynamics(rbdlmodel, q_np, qdot_np, tau_np, qddot_np)
    

In [3]:
#pbmodel = pb.loadURDF("pantilt.urdf")
sim = pb.connect(pb.DIRECT)
#pbmodel = pb.loadURDF("/urdf4timing/1dof.urdf")
ndofs = 10
urdf_nr = list(range(ndofs + 1))
urdf_nr.pop(0)


nitr = 1000
nrepeat = 100
    

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf"
root = 'base_link'
tip = "link16"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
asd = u2c.URDFparser()
robot_desc = asd.from_file(path_to_urdf)

jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = asd.get_n_joints(root, tip)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
c_rbdl = np.zeros(njoints)
ID_rbdl = np.zeros(njoints)

#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints
g_pb = [None]*njoints

qddot = []
for i in range(njoints):
    qddot.append(0.)

qdot = []
for i in range(njoints):
    qdot.append(0.)

    
        
M_kdl = kdl.JntSpaceInertiaMatrix(njoints)
M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)
tau_np = np.zeros(njoints)
tau_none = [None]*njoints

g_u2c = asd.get_gravity_rnea(root, tip, gravity_u2c)
c_u2c = asd.get_coriolis_rnea(root, tip)
M_u2c = asd.get_inertia_matrix_crba(root, tip)
ID_u2c = asd.get_inverse_dynamics_rnea(root, tip, gravity_u2c)
aba_u2c = asd.get_forward_dynamics_aba(root, tip, gravity_u2c)
crba_u2c = asd.get_forward_dynamics_crba(root, tip, gravity_u2c)


/home/lmjohann/urdf2casadi/examples/urdf/snake_robot.urdf


In [4]:

timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c  = median(timeit_g_u2c)/nitr*1000000

timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(timeit_g_pb)/nitr*1000000

timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(timeit_g_rbdl)/nitr*1000000

timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(timeit_g_kdl)/nitr*1000000



timeit_c_u2c = repeat("c_u2c_func()", setup = "from __main__ import c_u2c_func", repeat = nrepeat, number = nitr)
mediantime_c_u2c  = median(timeit_c_u2c)/nitr*1000000

timeit_c_pb = repeat("c_pb_func()", setup = "from __main__ import c_pb_func", repeat = nrepeat, number = nitr)
mediantime_c_pb = median(timeit_c_pb)/nitr*1000000

timeit_c_rbdl = repeat("c_rbdl_func()", setup = "from __main__ import c_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_c_rbdl = median(timeit_c_rbdl)/nitr*1000000

timeit_c_kdl = repeat("c_kdl_func()", setup = "from __main__ import c_kdl_func", repeat = nrepeat, number = nitr)
mediantime_c_kdl = median(timeit_c_kdl)/nitr*1000000


timeit_id_u2c = repeat("ID_u2c_func()", setup = "from __main__ import ID_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c  = median(timeit_id_u2c)/nitr*1000000

timeit_id_pb = repeat("ID_pb_func()", setup = "from __main__ import ID_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(timeit_id_pb)/nitr*1000000

timeit_id_rbdl = repeat("ID_rbdl_func()", setup = "from __main__ import ID_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(timeit_id_rbdl)/nitr*1000000


timeit_M_u2c = repeat("M_u2c_func()", setup = "from __main__ import M_u2c_func", repeat = nrepeat, number = nitr)
mediantime_M_u2c  = median(timeit_M_u2c)/nitr*1000000

timeit_M_pb = repeat("M_pb_func()", setup = "from __main__ import M_pb_func", repeat = nrepeat, number = nitr)
mediantime_M_pb = median(timeit_M_pb)/nitr*1000000

timeit_M_rbdl = repeat("M_rbdl_func()", setup = "from __main__ import M_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_M_rbdl = median(timeit_M_rbdl)/nitr*1000000

timeit_M_kdl = repeat("M_kdl_func()", setup = "from __main__ import M_kdl_func", repeat = nrepeat, number = nitr)
mediantime_M_kdl = median(timeit_M_kdl)/nitr*1000000


timeit_fdaba_u2c = repeat("aba_u2c_func()", setup = "from __main__ import aba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdaba_u2c = median(timeit_fdaba_u2c)/nitr*1000000

timeit_fdcrba_u2c = repeat("crba_u2c_func()", setup = "from __main__ import crba_u2c_func", repeat = nrepeat, number = nitr)
mediantime_fdcrba_u2c = median(timeit_fdcrba_u2c)/nitr*1000000

timeit_fd_rbdl = repeat("fd_rbdl_func()", setup = "from __main__ import fd_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_fd_rbdl = median(timeit_fd_rbdl)/nitr*1000000


In [5]:
print "\nsnake_g_u2c=", mediantime_g_u2c
print "\nsnake_id_u2c=", mediantime_id_u2c
print "\nsnake_C_u2c=", mediantime_c_u2c
print "\nsnake_M_u2c=", mediantime_M_u2c

print "\nsnake_g_rbdl=", mediantime_g_rbdl
print "\nsnake_id_rbdl=", mediantime_id_rbdl
print "\nsnake_C_rbdl=", mediantime_c_rbdl
print "\nsnake_M_rbdl=", mediantime_M_rbdl

print "\nsnake_g_pb=", mediantime_g_pb
print "\nsnake_id_pb=", mediantime_id_pb
print "\nsnake_C_pb=", mediantime_c_pb
print "\nsnake_M_pb=", mediantime_M_pb

print "\nsnake_g_kdl=", mediantime_g_kdl
print "\nsnake_C_kdl=", mediantime_c_kdl
print "\nsnake_M_kdl=", mediantime_M_kdl

print "\fdrbdl=", mediantime_fd_rbdl
print "\nfdcrba=", mediantime_fdcrba_u2c 
print "\nfdaba=", mediantime_fdaba_u2c


snake_g_u2c= 22.607922554

snake_id_u2c= 54.6989440918

snake_C_u2c= 38.0384922028

snake_M_u2c= 23.862361908

snake_g_rbdl= 14.088511467

snake_id_rbdl= 29.7459363937

snake_C_rbdl= 21.2104320526

snake_M_rbdl= 14.1924619675

snake_g_pb= 11.3346576691

snake_id_pb= 25.2009630203

snake_C_pb= 18.262386322

snake_M_pb= 15.8640146255

snake_g_kdl= 22.5094556808

snake_C_kdl= 31.4583778381

snake_M_kdl= 25.65741539
drbdl= 33.0110788345

fdcrba= 71.441411972

fdaba= 54.6615123749


In [ ]:

snake_g_u2c= 22.607922554

snake_id_u2c= 54.6989440918

snake_C_u2c= 38.0384922028

snake_M_u2c= 23.862361908

snake_g_rbdl= 14.088511467

snake_id_rbdl= 29.7459363937

snake_C_rbdl= 21.2104320526

snake_M_rbdl= 14.1924619675

snake_g_pb= 11.3346576691

snake_id_pb= 25.2009630203

snake_C_pb= 18.262386322

snake_M_pb= 15.8640146255

snake_g_kdl= 22.5094556808

snake_C_kdl= 31.4583778381

snake_M_kdl= 25.65741539
drbdl= 33.0110788345

fdcrba= 71.441411972

fdaba= 54.6615123749


# Derivatives 

## q

In [6]:
import casadi as cs

qs =cs.SX.sym("qs",njoints)
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)

C_j = cs.jacobian(c_u2c(qs, qsdot), qs)
g_j = cs.jacobian(g_u2c(qs), qs)
M_j = cs.jacobian(M_u2c(qs), qs)
invd_j = cs.jacobian(ID_u2c(qs, qsdot, qsddot), qs)
fd_aba_j = cs.jacobian(aba_u2c(qs, qsdot, taus), qs)
fd_crba_j = cs.jacobian(crba_u2c(qs, qsdot, taus), qs)


g_j = cs.Function("g_j", [qs], [g_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
C_j = cs.Function("C_j", [qs, qsdot], [C_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
M_j = cs.Function("M_j", [qs], [M_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
invd_j = cs.Function("invd_j", [qs, qsdot, qsddot], [invd_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [7]:


n_invdj = invd_j.n_nodes()
n_Cj = C_j.n_nodes()
n_Mj = M_j.n_nodes()
n_gj = g_j.n_nodes()
n_fd_abaj = fd_aba_j.n_nodes()
n_fd_crbaj = fd_crba_j.n_nodes()

print "\nsnake_nodes_id_j=", n_invdj
print "\nsnake_nodes_c_d=", n_Cj
print "\nsnake_nodes_m_d=", n_Mj
print "\nsnake_nodes_g_d=", n_gj
print "\nsnake_nodes_fdaba_d", n_fd_abaj
print "\nsnake_nodes_fdcrba_d =",n_fd_crbaj



snake_nodes_id_j= 7128

snake_nodes_c_d= 6217

snake_nodes_m_d= 11635

snake_nodes_g_d= 2347

snake_nodes_fdaba_d 30719

snake_nodes_fdcrba_d = 417474


In [16]:
def fdcrba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_j(q_none, qdot_none, tau_none)

def fdaba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_j(q_none, qdot_none, tau_none)
    
    
def M_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2   
    M_j(q_none)
    
def g_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    g_j(q_none)


def C_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    C_j(q_none, qdot_none)
    
def ID_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    invd_j(q_none, qdot_none, qddot_none)

def aba_j_func():   
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_j(q_none, qdot_none, tau_none)
    
def crba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_crba_j(q_none, qdot_none, tau_none)

In [18]:
timeit_idj = repeat("ID_j_func()", setup = "from __main__ import ID_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000

timeit_gj = repeat("g_j_func()", setup = "from __main__ import g_j_func", repeat = nrepeat, number = nitr)
mediantime_gj = median(timeit_gj)/nitr*1000000

timeit_Cj = repeat("C_j_func()", setup = "from __main__ import C_j_func", repeat = nrepeat, number = nitr)
mediantime_Cj = median(timeit_Cj)/nitr*1000000

timeit_Mj = repeat("M_j_func()", setup = "from __main__ import M_j_func", repeat = nrepeat, number = nitr)
mediantime_Mj = median(timeit_Mj)/nitr*1000000

timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_g_d=", mediantime_gj
print "\nsnake_C_d=", mediantime_Cj
print "\nsnake_M_d=", mediantime_Mj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj


snake_id_d= 58.0686330795

snake_g_d= 24.8249769211

snake_C_d= 43.651342392

snake_M_d= 38.6494398117

snake_fdaba_d= 64.0840530396

snake_fdcrba_d= 69.7106122971




snake_id_d= 58.5190057755

snake_g_d= 24.9165296555

snake_C_d= 43.6835289001

snake_M_d= 38.4865999222

snake_fdaba_d= 63.8264417648

snake_fdcrba_d= 69.1299438477

## qdot 

In [10]:
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)

C_jdot = cs.jacobian(c_u2c(qs, qsdot), qsdot)
invd_jdot = cs.jacobian(ID_u2c(qs, qsdot, qsddot), qsdot)
fd_aba_jdot = cs.jacobian(aba_u2c(qs, qsdot, taus), qsdot)
fd_crba_jdot = cs.jacobian(crba_u2c(qs, qsdot, taus), qsdot)


C_jdot = cs.Function("C_j", [qs, qsdot], [C_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
invd_jdot = cs.Function("invd_j", [qs, qsdot, qsddot], [invd_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_jdot = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_jdot = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_jdot], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [19]:
def fdcrba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_crba_jdot(q_none, qdot_none, tau_none)

def fdaba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    fd_aba_jdot(q_none, qdot_none, tau_none)
    
    
    
#def C_j_func():
#    for j in range(njoints):
#        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
#        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
#    C_j(q_none, qdot_none)


def C_j_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    C_jdot(q_none, qdot_none)
    
def ID_j_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
 
    invd_jdot(q_none, qdot_none, qddot_none)
    
#def id_j_func():
#    for j in range(njoints):
#        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
#        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
#        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
#    invd_j(q_none, qdot_none, qddot_none)


def aba_j_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    fd_aba_jdot(q_none, qdot_none, tau_none)
    
def crba_j_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        tau_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    fd_crba_jdot(q_none, qdot_none, tau_none)

In [21]:
timeit_idj = repeat("ID_j_func()", setup = "from __main__ import ID_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000


timeit_Cj = repeat("C_j_func()", setup = "from __main__ import C_j_func", repeat = nrepeat, number = nitr)
mediantime_Cj = median(timeit_Cj)/nitr*1000000


timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_C_d=", mediantime_Cj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj


snake_id_d= 54.5409917831

snake_C_d= 37.0849370956

snake_fdaba_d= 58.3474636078

snake_fdcrba_d= 73.3815431595


In [ ]:

snake_id_d= 53.680062294

snake_C_d= 36.3329648972

snake_fdaba_d= 57.6950311661

snake_fdcrba_d= 73.1304883957

## qddot / tau

In [14]:
qsdot =cs.SX.sym("qsdot",njoints)
qsddot =cs.SX.sym("qsddot",njoints)
taus =cs.SX.sym("taus",njoints)

invd_j = cs.jacobian(ID_u2c(qs, qsdot, qsddot), qsddot)
fd_aba_j = cs.jacobian(aba_u2c(qs, qsdot, taus), taus)
fd_crba_j = cs.jacobian(crba_u2c(qs, qsdot, taus), taus)

invd_j = cs.Function("invd_j", [qs, qsdot, qsddot], [invd_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_aba_j = cs.Function("fd_aba_j", [qs, qsdot, taus], [fd_aba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})
fd_crba_j = cs.Function("fd_crba_j", [qs, qsdot, taus], [fd_crba_j], {"jit": True, "jit_options":{"flags":"-Ofast"}})

In [23]:
timeit_idj = repeat("ID_j_func()", setup = "from __main__ import ID_j_func", repeat = nrepeat, number = nitr)
mediantime_idj = median(timeit_idj)/nitr*1000000


timeit_fdabaj = repeat("aba_j_func()", setup = "from __main__ import aba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_abaj = median(timeit_fdabaj)/nitr*1000000

timeit_fdcrbaj = repeat("crba_j_func()", setup = "from __main__ import crba_j_func", repeat = nrepeat, number = nitr)
mediantime_fd_crbaj = median(timeit_fdcrbaj)/nitr*1000000

   
print "\nsnake_id_d=", mediantime_idj
print "\nsnake_C_d=", mediantime_Cj
print "\nsnake_fdaba_d=", mediantime_fd_abaj
print "\nsnake_fdcrba_d=", mediantime_fd_crbaj


snake_id_d= 54.8444986343

snake_C_d= 37.0849370956

snake_fdaba_d= 58.310508728

snake_fdcrba_d= 73.4159946442


In [ ]:

snake_id_d= 54.7025203705

snake_fdaba_d= 58.310508728

snake_fdcrba_d= 73.4159946442